### Boiler Plate Code

In [1]:
import random
import os
from time import time

import numpy as np
from tqdm import tqdm
from QAP_Heuristic import QAP_Heuristic

import matplotlib.pyplot as plt
%matplotlib inline

### Simmulated Annealing 

In [2]:
class SimmulatedAnnealing(QAP_Heuristic):
    def __init__(self, w, d, rate, Tmin, T0) -> None:
        super().__init__(w, d)
        self.rate = rate
        self.Tmin = Tmin
        self.T0   = T0

    def __str__():
        return "SimmulatedAnnealing"


    def solve(self, n_iter):
        
        # initialise a random solution 
        X = np.array(list(range(self.n)))
        np.random.shuffle(X)
        
        # code for simmulated annealing
        perm = X
        curr_cost = self.cost(perm)
        best_cost = curr_cost
        best_perm = perm
        T = self.T0
        
        best_cost_history = [best_cost]

        while T > self.Tmin:
            
            if time() > self.MAX_CPU_TIME: break

            for _ in range(n_iter):

                if time() > self.MAX_CPU_TIME: break
                
                new_perm = self.gen_neighbour(perm)
                new_cost = self.cost(new_perm)
                
                ΔE = new_cost - curr_cost
                if (ΔE <= 0) or (np.exp( -ΔE/T ) >= np.random.uniform()):
                    perm = new_perm
                    curr_cost = new_cost
                    
                    if curr_cost < best_cost:
                        best_cost = curr_cost
                        best_perm = perm
                        
    
                best_cost_history.append(best_cost)
        
            T *= self.rate
            
        return best_perm, best_cost, best_cost_history
    
    @staticmethod
    def gen_neighbour(perm):
    # randomly generates a neighbor through swapping two indices
    # with probability 10%, we shuffle the entire array and return

        new_perm = perm[:]
        # Two random indexes
        i = random.randint(0, len(perm)-1)
        j = random.randint(0, len(perm)-1)

        new_perm[i], new_perm[j] = new_perm[j], new_perm[i]   

        if random.random() < 0.1:
            np.random.shuffle(new_perm)

        return new_perm

### Automated Testing

Tests simmulated annealing on QAPLib instances

In [5]:
import csv

instance_path = '../QAPInstances/'
soln_path     = '../QAPSolns/'

# for managing file opening and closing
def read_integers(filename):
    with open(filename) as f:
        return [int(elem) for elem in f.read().split()]


def open_solution(filename: str):
    file_it = iter(read_integers(filename))
    _ = next(file_it)    # this is just how the files within the lib are formatted
    return next(file_it)

        

def test_hueristic(n_iters=10_000, tai_only=False, bur_only=False):
    """
    n_iters: number of iterations on each test instance
    tai_only: if True, restricts testing to the Tai instances, reducing computation time
    """
    
    results_filename = '../results/SimmulatedAnnleaing.csv'
    if bur_only: results_filename = '../results/SimmulatedAnnleaing-bur.csv'
    if tai_only: results_filename = '../results/SimmulatedAnnleaing-tai.csv'
    with open(results_filename, mode='a') as f:
        writer = csv.writer(f)
        for filename in os.listdir(instance_path):
            
            # skips any instances that are not Tai
            if tai_only and 'tai' not in filename: continue  
            if bur_only and 'bur' not in filename: continue  

            file_it = iter(read_integers(instance_path+filename))

            # open QAP instance param's 
            n = next(file_it)
            w = np.array([[next(file_it) for j in range(n)] for i in range(n)])
            d = np.array([[next(file_it) for j in range(n)] for i in range(n)])

            # generate an instance
            heuristic = SimmulatedAnnealing(w, d, rate=0.98,Tmin=1_000,T0=100_000_000)

            # open up corresponding soln from QAPLib: 
            soln_file = filename[:-4]+'.sln' # this removes the .dat from filename
            
            try:
                qap_soln = open_solution(soln_path+soln_file)
                _, huerstic_cost, _ = heuristic.solve(n_iters)

                # compute gap
                gap = 100*(huerstic_cost - qap_soln)/qap_soln
                
                print(filename, gap)
                writer.writerow([filename, gap])
                

            # any instances without corresponding solution files are deleted
            except FileNotFoundError:
                os.remove(instance_path+filename)

    return

In [6]:
test_hueristic(n_iters=100_000, bur_only=True)

bur26h.dat 2.1154984505522028
bur26e.dat 1.4322393356152978
bur26d.dat 2.159726265791729
bur26f.dat 2.397645294449245
bur26g.dat 1.7545416841781478
bur26c.dat 2.0203269148733276
bur26b.dat 2.021398419844457
bur26a.dat 1.7780517333834562
